In [1]:
import pandas as pd
from pathlib import Path

processed_path = Path("../data/processed/corn_exports_2025.csv")
weekly_data = pd.read_csv(processed_path, parse_dates=["date"])
weekly_data.head()

,date,weekly_exports_mt,weekly_percent_change,moving_average_4w,std_4w
0,2025-09-04,687326,NaN,NaN,NaN
1,2025-09-11,1555901,1.263702,NaN,NaN
2,2025-09-18,1315867,-0.154273,NaN,NaN
3,2025-09-25,1429180,0.086113,1247068.5,385826.822723
4,2025-10-02,1830620,0.280888,1532892.0,221380.069589


In [7]:
# Some summary statistics
total_exports = weekly_data["weekly_exports_mt"].sum()
high_week = weekly_data.loc[weekly_data["weekly_exports_mt"].idxmax()]
low_week = weekly_data.loc[weekly_data["weekly_exports_mt"].idxmin()]
avg_weekly = weekly_data["weekly_exports_mt"].mean()
